In [1]:
from harmony_tokenizers_m21 import ChordSymbolTokenizer, PitchClassTokenizer, MelodyPitchTokenizer, MergedMelHarmTokenizer
from data_utils import StructBARTMelHarmDataset
from torch.utils.data import DataLoader
import torch
from transformers import BartForConditionalGeneration, BartConfig, DataCollatorForSeq2Seq
import numpy as np

from a_star import AStarBART

/home/maximos/miniconda3/envs/torch/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/media/maindisk/maximos/repos/CMelHarmTransformer/a_star.py:27: SyntaxWarning: invalid escape sequence '\m'
  or '<\m>' in tokens[i]:


In [2]:
test_dir = '/media/maindisk/maximos/data/hooktheory_test'

cstok = ChordSymbolTokenizer()
pctok = PitchClassTokenizer()
meltok = MelodyPitchTokenizer()
tokenizer = MergedMelHarmTokenizer(meltok, cstok)
# tokenizer = MergedMelHarmTokenizer(meltok, pctok)

test_dataset = StructBARTMelHarmDataset(test_dir, tokenizer, max_length=512, num_bars=16)

In [3]:
bart_config = BartConfig(
    vocab_size=len(tokenizer.vocab),
    pad_token_id=tokenizer.pad_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    decoder_start_token_id=tokenizer.bos_token_id,
    forced_eos_token_id=tokenizer.eos_token_id,
    max_position_embeddings=512,
    encoder_layers=8,
    encoder_attention_heads=8,
    encoder_ffn_dim=512,
    decoder_layers=8,
    decoder_attention_heads=8,
    decoder_ffn_dim=512,
    d_model=512,
    encoder_layerdrop=0.25,
    decoder_layerdrop=0.25,
    dropout=0.25
)

model = BartForConditionalGeneration(bart_config)

model_path = 'saved_models/bart/ChordSymbolTokenizer/ChordSymbolTokenizer.pt'
# model_path = 'saved_models/bart/PitchClassTokenizer/PitchClassTokenizer.pt'

# device_name = 'cuda:0'
device_name = 'cpu'
if device_name == 'cpu':
    device = torch.device('cpu')
else:
    if torch.cuda.is_available():
        device = torch.device(device_name)
    else:
        print('Selected device not available: ' + device_name)

checkpoint = torch.load(model_path, map_location=device_name, weights_only=True)
model.load_state_dict(checkpoint)
model.eval()
model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(547, 512, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(547, 512, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(514, 512)
      (layers): ModuleList(
        (0-7): 8 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=512, out_features=512, bias=True)
          (fc2): Linear(in_features=512, out_features=512, bias=True)
          (final_layer_norm

In [4]:
def create_data_collator(tokenizer, model):
    return DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding=True)

collator = create_data_collator(tokenizer, model=model)

trainloader = DataLoader(test_dataset, batch_size=1, shuffle=True, collate_fn=collator)

In [5]:
batch = next(iter(trainloader))

/home/maximos/miniconda3/envs/torch/lib/python3.12/site-packages/music21/stream/base.py:3694: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  return self.iter().getElementsByClass(classFilterList)
/home/maximos/miniconda3/envs/torch/lib/python3.12/site-packages/transformers/data/data_collator.py:656: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


In [6]:
print(batch)

{'input_ids': tensor([[  2,   6, 183,  98,  58, 106,  61, 114,  61, 122,  58, 126,  56,   6,
          98,  58, 106,  61, 114,  61, 122,  58, 126,  56,   6,  98,  58, 106,
          63, 114,  63, 122,  58, 126,  56,   6,  98,  58, 106,  63, 114,  63,
         122,   4, 126,  58, 128,  58,   6,  98,  61, 106,  65, 114,  65, 122,
          63, 126,  61,   6,  98,  61, 106,  65, 114,  65, 122,  63, 126,  65,
           6,  98,  66, 114,  61,   6,  98,  57, 114,   4, 122,  61, 126,  63,
           8,   6,   9,   6,   9,   6,   9,   6,   9,   6,   9,   6,   9,   6,
          98, 355,   9,   6,   9,   3]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': ten

In [7]:
all_ids = batch['input_ids']
print(all_ids)
melody_end_index = all_ids[0].tolist().index( tokenizer.vocab['</m>'] )
print(melody_end_index)
constraint_ids = all_ids[0][melody_end_index:]
print(constraint_ids)
# start_harmony_position = np.where( all_ids == harmony_start_index )[0][0]
input_ids = all_ids.clone()
print(input_ids.shape)
print(tokenizer.vocab['</s>'])

tensor([[  2,   6, 183,  98,  58, 106,  61, 114,  61, 122,  58, 126,  56,   6,
          98,  58, 106,  61, 114,  61, 122,  58, 126,  56,   6,  98,  58, 106,
          63, 114,  63, 122,  58, 126,  56,   6,  98,  58, 106,  63, 114,  63,
         122,   4, 126,  58, 128,  58,   6,  98,  61, 106,  65, 114,  65, 122,
          63, 126,  61,   6,  98,  61, 106,  65, 114,  65, 122,  63, 126,  65,
           6,  98,  66, 114,  61,   6,  98,  57, 114,   4, 122,  61, 126,  63,
           8,   6,   9,   6,   9,   6,   9,   6,   9,   6,   9,   6,   9,   6,
          98, 355,   9,   6,   9,   3]])
84
tensor([  8,   6,   9,   6,   9,   6,   9,   6,   9,   6,   9,   6,   9,   6,
         98, 355,   9,   6,   9,   3])
torch.Size([1, 104])
3


In [8]:
astar = AStarBART( model, tokenizer, input_ids, constraint_ids, max_length=512, beam_width=20, lookahead_k=10 )

In [9]:
print(astar.constraint_bar)
print(astar.position_token)
print(astar.chord_tokens)

7
position_0x00
['F:min6']


In [10]:
generated_ids, model_calls = astar.decode()
generated_tokens = []

In [11]:
print(generated_ids.shape)

torch.Size([1, 17])


In [12]:
print(model_calls)

114


In [13]:
for i in generated_ids[0]:
    generated_tokens.append( tokenizer.ids_to_tokens[ int(i) ].replace(' ','x') )
print(generated_tokens)

['<s>', '<h>', '<bar>', '<bar>', '<bar>', '<bar>', '<bar>', 'position_0x00', 'C:maj', '<bar>', '<bar>', 'position_0x00', 'F:min6', '<bar>', 'position_0x00', 'F:min6', '</s>']


In [14]:
t = generated_tokens
line = []
for i in range(len(t)):
    line.append(t[i])
    if i+1 < len(t) and 'bar' in t[i+1]:
        print(line)
        line = []
print(line)

['<s>', '<h>']
['<bar>']
['<bar>']
['<bar>']
['<bar>']
['<bar>', 'position_0x00', 'C:maj']
['<bar>']
['<bar>', 'position_0x00', 'F:min6']
['<bar>', 'position_0x00', 'F:min6', '</s>']
